# CHALLENGE 4
##  Analyze data

- Query table "vehicles_enriched" in gold layer
- Aggregate data by municipality_name (array)
- Calculate:
  - count of vehicles (id) that pass through that municipality
  - sum speed of vehicles

Questions:
  - What are the top 3 municipalities by vehicles routes?
  - What are the top 3 municipalities with higher vehicle speed on average?


Tips:
- explode array into rows -> https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html


# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Challenge 4').config('spark.ui.port', '4050').getOrCreate()

In [2]:
# Create folder for bronze layer
!mkdir -p /content/lake/bronze
print('Bronze layer created')

Bronze layer created


In [3]:
# Create folder for silver layer
!mkdir -p /content/lake/silver
print('Silver layer created')

Silver layer created


In [4]:
# Create folder for gold layer
!mkdir -p /content/lake/gold
print('Gold layer created')

Gold layer created


# Define ETLFlow Class
API Ingestion and Load class

In [5]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import requests

class ETLFlow:
    def __init__(self, spark: SparkSession) -> None:
        self.spark = spark

    # Extract data from API endpoint
    # Parameters:
    #   url: The API endpoint URL.
    #   schema: Optional schema to enforce on the JSON data
    # Returns:
    #   DataFrame containing the extracted data.
    def ReadAPI(self, url: str, schema: StructType = None) -> DataFrame:
        response = requests.get(url)  # Fetch data from the API
        rdd = self.spark.sparkContext.parallelize(response.json())  # Convert JSON response to RDD
        if schema:
            df = self.spark.read.schema(schema).json(rdd)  # Read RDD into DataFrame with schema
        else:
            df = self.spark.read.json(rdd)  # Read RDD into DataFrame without schema
        return df


    # Load a DataFrame into storage in the specified format.
    # Parameters:
    #   df: The DataFrame to be saved.
    #   format: The format to save the DataFrame in (e.g., parquet).
    #   path: The path where the DataFrame should be saved.
    #   partition_column: Optional column to partition the data by.
    def load(self, df: DataFrame, format: str, path: str, partition_column: str = None) -> None:
        if partition_column:
            # Save as a single file, partitioned by the specified column
            df.coalesce(1).write.mode("overwrite").partitionBy(partition_column).format(format).save(path)
        else:
            # Save as a single file without partitioning
            df.coalesce(1).write.mode("overwrite").format(format).save(path)

# ETL Task Ingestion for Bronze layer

---

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

class ETLTask_Bronze(ETLFlow):
    def __init__(self, spark: SparkSession) -> None:
        # Initialize ETLTask by inheriting from ETLFlow
        super().__init__(spark)


    # Ingest vehicle data from the API and load it into the bronze layer.
    #   Extract data from 'vehicles' endpoint.
    #   Add 'date' column derived from 'timestamp'.
    #   Saves data as parquet, partitioned by 'date'.
    def ingestion_vehicles(self):
        # Define schema for vehicle
        vehicle_schema = StructType([
            StructField('bearing', IntegerType(), True),
            StructField('block_id', StringType(), True),
            StructField('current_status', StringType(), True),
            StructField('id', StringType(), True),
            StructField('lat', FloatType(), True),
            StructField('line_id', StringType(), True),
            StructField('lon', FloatType(), True),
            StructField('pattern_id', StringType(), True),
            StructField('route_id', StringType(), True),
            StructField('schedule_relationship', StringType(), True),
            StructField('shift_id', StringType(), True),
            StructField('speed', FloatType(), True),
            StructField('stop_id', StringType(), True),
            StructField('timestamp', TimestampType(), True),
            StructField('trip_id', StringType(), True)
        ])

        # Extract data using defined schema
        df = self.ReadAPI(url="https://api.carrismetropolitana.pt/vehicles", schema=vehicle_schema)

        # Create "date" column from "timestamp"
        df = df.withColumn("date", date_format(col("timestamp"), "yyyy-MM-dd"))
        #df.show()

        # Load data into the bronze layer, partitioned by "date"
        self.load(df=df, format="parquet", path="/content/lake/bronze/vehicles", partition_column="date")


    # Ingest line data from the API and load it into the bronze layer.
    #   Extracts data from 'lines' endpoint.
    #   Saves data as parquet without partitioning.
    def ingestion_lines(self):
        # Define schema for lines
        lines_schema = StructType([
            StructField('_corrupt_record', StringType(), True),
            StructField('color', StringType(), True),
            StructField('facilities', ArrayType(StringType()), True),
            StructField('id', StringType(), True),
            StructField('localities', ArrayType(StringType()), True),
            StructField('long_name', StringType(), True),
            StructField('municipalities', ArrayType(StringType()), True),
            StructField('patterns', ArrayType(StringType()), True),
            StructField('routes', ArrayType(StringType()), True),
            StructField('short_name', StringType(), True),
            StructField('text_color', StringType(), True)
        ])

        # Extract data using defined schema
        df = self.ReadAPI(url="https://api.carrismetropolitana.pt/lines", schema=lines_schema)
        #df.show()

        # Load data into the bronze layer
        self.load(df=df, format="parquet", path="/content/lake/bronze/lines")


    # Ingest municipality data from the API and load it into the bronze layer.
    #   Extracts data from 'municipalities' endpoint.
    #   Saves data as parquet without partitioning.
    def ingestion_municipalities(self):
       # Define schema for municipalities
       municipalities_schema = StructType([
           StructField('district_name', StringType(), True),
           StructField('id', StringType(), True),
           StructField('name', StringType(), True),
           StructField('prefix', StringType(), True),
           StructField('region_id', StringType(), True),
           StructField('region_name', StringType(), True)
       ])


       # Extract data using defined schema
       df = self.ReadAPI(url="https://api.carrismetropolitana.pt/municipalities", schema=municipalities_schema)
       #df.show()

       # Load data into the bronze layer
       self.load(df=df, format="parquet", path="/content/lake/bronze/municipalities")

print('ETLTask class defined for specific API endpoint ingestion')

ETLTask class defined for specific API endpoint ingestion


# ETL Task Cleansing for Silver layer

---------------------

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

class ETLTask_Silver(ETLFlow):
    def __init__(self, spark: SparkSession) -> None:
        super().__init__(spark)

    def cleanse_vehicles(self):
        # Read data from bronze layer
        df = self.spark.read.parquet("/content/lake/bronze/vehicles")

        # Apply vehicle transformations:
        #   Changes column name "lat" to "latitude"
        #   Changes column name "lon" to "longitude"
        #   Removes duplicate records
        #   Remove rocords where "current_status" has null values
        #   Remove corrupted records - This removes any rows that contain null values in any column
        df = df.withColumnRenamed("lat", "latitude") \
               .withColumnRenamed("lon", "longitude") \
               .dropDuplicates() \
               .filter(col("current_status").isNotNull()) \
               .na.drop()

        df.show()

        # Write to silver layer
        self.load(df=df, format="parquet", path="/content/lake/silver/vehicles", partition_column="date")

    def cleanse_lines(self):
        # Read data from bronze layer
        df = self.spark.read.parquet("/content/lake/bronze/lines")

        # Apply lines transformations:
        #   Removes duplicate records
        #   Remove rocords that have values in "_corrupt_record" column
        #   Remove "_corrupt_record" column
        #   Remove any corrupted records - Removes any rows that contain null values in any column
        df = df.dropDuplicates() \
               .filter(col("_corrupt_record").isNull()) \
               .drop("_corrupt_record") \
               .na.drop()

        df.show()

        # Write to silver layer
        self.load(df=df, format="parquet", path="/content/lake/silver/lines")

    def cleanse_municipalities(self):
        # Read data from bronze layer
        df = self.spark.read.parquet("/content/lake/bronze/municipalities")

        # Apply municipalities transformations:
        #   Removes duplicate records
        #   Only returns values where "name" and "district_name" aren't null
        #   Remove any corrupted records - Removes any rows that contain null values in any column
        df = df.dropDuplicates() \
               .filter(col("name").isNotNull() & col("district_name").isNotNull()) \
               .na.drop()

        df.show()

        # Write to silver layer
        self.load(df=df, format="parquet", path="/content/lake/silver/municipalities")

print('ETLTask class defined for bronze layer cleansing')

ETLTask class defined for bronze layer cleansing


# ETL Task Enrichment for Gold layer

---------------------

In [8]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, explode
from pyspark.sql.types import *

class ETLTask_Gold(ETLFlow):
    def __init__(self, spark: SparkSession) -> None:
        super().__init__(spark)

    def enrich_vehicles(self):
        # Define schemas
        vehicles_schema = StructType([
            StructField('bearing', IntegerType(), True),
            StructField('block_id', StringType(), True),
            StructField('current_status', StringType(), True),
            StructField('id', StringType(), True),
            StructField('latitude', FloatType(), True),
            StructField('line_id', StringType(), True),
            StructField('longitude', FloatType(), True),
            StructField('pattern_id', StringType(), True),
            StructField('route_id', StringType(), True),
            StructField('schedule_relationship', StringType(), True),
            StructField('shift_id', StringType(), True),
            StructField('speed', FloatType(), True),
            StructField('stop_id', StringType(), True),
            StructField("timestamp", TimestampType(), True),
            StructField('trip_id', StringType(), True),
            StructField("date", DateType(), True)
        ])

        lines_schema = StructType([
            StructField("id", StringType(), True),
            StructField("long_name", StringType(), True),
            StructField("municipalities", ArrayType(StringType()), True)
        ])

        municipalities_schema = StructType([
            StructField("id", StringType(), True),
            StructField("name", StringType(), True)
        ])

        # Read data from silver layer
        vehicles_df = self.spark.read.schema(vehicles_schema).parquet("/content/lake/silver/vehicles")
        lines_df = self.spark.read.schema(lines_schema).parquet("/content/lake/silver/lines")
        municipalities_df = self.spark.read.schema(municipalities_schema).parquet("/content/lake/silver/municipalities")

        # Join vehicles with lines (1:1)
        enriched_df = vehicles_df.join(lines_df, vehicles_df.line_id == lines_df.id, "left")

        # Explode municipalities before joining to get the same cardinality as lines df (1:n)
        exploded_lines_df = lines_df.select(col("id").alias("line_id"), explode("municipalities").alias("municipality_id"))

        # Join with exploded municipalities and then with municipalities_df
        enriched_df = enriched_df.join(exploded_lines_df, enriched_df.line_id == exploded_lines_df.line_id, "left") \
                                 .join(municipalities_df, exploded_lines_df.municipality_id == municipalities_df.id, "left")

        # Select required columns
        enriched_df = enriched_df.select(
            vehicles_df["*"],
            col("long_name").alias("line_name"),
            col("name").alias("municipality_name")
        )

        enriched_df.show()

        # Write enriched data to gold layer
        self.load(df=enriched_df, format="parquet", path="/content/lake/gold/vehicles_enriched", partition_column="date")

#Main Execution

In [9]:
# Initialize ETL Bronze Process
print("Starting ETL program")
etl_bronze = ETLTask_Bronze(spark)

# Ingest vehicle data
print("Running Task - Ingestion Vehicles")
etl_bronze.ingestion_vehicles()

# Ingest line data
print("Running Task - Ingestion Lines")
etl_bronze.ingestion_lines()

# Ingest municipality data
print("Running Task - Ingestion Municipalities")
etl_bronze.ingestion_municipalities()

Starting ETL program
Running Task - Ingestion Vehicles
Running Task - Ingestion Lines
Running Task - Ingestion Municipalities


In [10]:
# Initialize ETL Silver Process
print("Starting Cleansing process")
etl_silver = ETLTask_Silver(spark)
print("Ingestion process completed")

# Cleanse vehicle data
print("Running Task - Cleansing Vehicles")
etl_silver.cleanse_vehicles()

# Cleanse lines data
print("Running Task - Cleansing Lines")
etl_silver.cleanse_lines()

# Cleanse municipalities data
print("Running Task - Cleansing Municipalities")
etl_silver.cleanse_municipalities()

print("Cleansing process completed")

Starting Cleansing process
Ingestion process completed
Running Task - Cleansing Vehicles
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+----------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|      date|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+----------+
|    135|       ESC_DU_EU1002| IN_TRANSIT_TO| 43|2391|  38.6158|   3103|-9.110255|  3103_0_1|  3103_0|            SCHEDULED|      EU1076|1.3888888| 140225|2024-11-27 17:21:17|3103_0_1_1630_165...|2024-11-27|
|    147|             1034-11| IN_TRANSIT_TO| 42|2103|   38.807|   2725| -9.115

In [11]:
# Initialize ETL Gold Process
etl_gold = ETLTask_Gold(spark)

print("Running Task - Enriching Vehicles")
etl_gold.enrich_vehicles()

print("Enrichment process completed")
print("ETL program completed")

Running Task - Enriching Vehicles
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+----------+--------------------+-----------------+
|bearing|            block_id|current_status|      id| latitude|line_id|longitude|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|      date|           line_name|municipality_name|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+----------+--------------------+-----------------+
|    135|       ESC_DU_EU1002| IN_TRANSIT_TO| 43|2391|  38.6158|   3103|-9.110255|  3103_0_1|  3103_0|            SCHEDULED|      EU1076|1.3888888| 140225|2024-11-27 17:21:17|3103_0_1_1630_165...|2024-11-27|Corroios (Estação.

#Data Analysis

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, count, sum, avg, desc


# Read the enriched data from the gold layer
df = spark.read.parquet("/content/lake/gold/vehicles_enriched")

# Explode the municipality_name array and aggregate data
# df.select(explode("municipality_name").alias("municipality"), "id", "speed") \
aggregated_df = df.select(col("municipality_name").alias("municipality"), "id", "speed") \
    .groupBy("municipality") \
    .agg(
        count("id").alias("vehicle_count"),
        sum("speed").alias("total_speed"),
        avg("speed").alias("avg_speed")
    )


# Question 1: Top 3 municipalities by vehicle routes
print("Question 1")
print("Top 3 municipalities by vehicles routes:")
aggregated_df.orderBy(desc("vehicle_count")).select("municipality", "vehicle_count").show(3)

# Question 2: Top 3 municipalities with higher vehicle speed on average
print("Question 2")
print("Top 3 municipalities with higher vehicle speed on average:")
aggregated_df.orderBy(desc("avg_speed")).select("municipality", "avg_speed").show(3)

Question 1
Top 3 municipalities by vehicles routes:
+------------+-------------+
|municipality|vehicle_count|
+------------+-------------+
|      Lisboa|          358|
|      Sintra|          243|
|      Loures|          199|
+------------+-------------+
only showing top 3 rows

Question 2
Top 3 municipalities with higher vehicle speed on average:
+-----------------+------------------+
|     municipality|         avg_speed|
+-----------------+------------------+
|Arruda dos Vinhos|10.277777671813965|
|          Montijo|  8.80824367461666|
|            Mafra| 8.260233920917177|
+-----------------+------------------+
only showing top 3 rows

